In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r drive/MyDrive/gym_examples/gym_examples .

In [ ]:
!ls

drive  gym_examples  sample_data


In [ ]:
!rm -f gym_examples

rm: cannot remove 'gym_examples': Is a directory


In [ ]:
!ls gym_examples

envs  __init__.py


In [ ]:
!ls gym_examples/

envs  __init__.py


In [ ]:
!ls gym_examples/envs

__init__.py  reversi_random.py	shooting_airplane.py


In [ ]:
!pip install gymnasium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 9.7 MB/s eta 0:00:00


In [ ]:
import gymnasium as gym
env = gym.make('gym_examples:gym_examples/Reversi-v0', render_mode='text')

NameNotFound: Environment `Reversi` doesn't exist in namespace gym_examples.

In [ ]:
obs, info = env.reset()

NameError: name 'env' is not defined

In [ ]:
env.render()

In [ ]:
info['action_mask']

In [ ]:
obs, reward, done, _, info = env.step((2, 4))

In [ ]:
reward

In [ ]:
env.render()

In [ ]:
info['action_mask']

In [ ]:
obs, reward, done, _,info = env.step((5, 3))

In [ ]:
reward

In [ ]:
env.render()

In [ ]:
info['action_mask']

In [ ]:
obs, reward, done,_, info = env.step((4, 6))
env.render()

In [ ]:
reward

dqn

In [ ]:
import numpy as np
import gymnasium as gym
import torch
import torch.nn as nn
import torchvision.transforms as T

# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = epsilon_max - epsilon_min # Rate at which to reduce chance
                                             # of random action being taken
batch_size = 16  # Size of batch taken from replay buffer
max_steps_per_episode = 60
max_episodes = 100

In [ ]:
import torch
import torch.nn as nn

num_actions = 64

class QModel(nn.Module):
    def __init__(self, num_actions):
        super(QModel, self).__init__()
        self.dropout = nn.Dropout(p=0.3)
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding='same')
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding='same')
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, stride=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(1152, 512)
        self.fc2 = nn.Linear(512, num_actions)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = self.dropout(x)
        x = nn.functional.relu(self.conv3(x))
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = self.dropout(x)
        action = self.fc2(x)
        return action

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# The first model makes the predictions for Q-values which are used to
# make a action.
model = QModel(num_actions)
model.to(device)

# Build a target model for the prediction of future rewards.
# The weights of a target model get updated every 10000 steps thus when the
# loss between the Q-values is calculated the target Q-value is stable.
model_target = QModel(num_actions)
model_target.to(device)

loss_function = nn.SmoothL1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00025)

In [ ]:
device

In [ ]:
# Experience replay buffers
action_history = []
action_mask_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 200000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 500000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000

In [ ]:
# Function to preprocess the state
# note that player 1 = env player, player 2 = agent
def preprocess_state(env_observ):
    st = torch.from_numpy(env_observ).squeeze()
    st = st.to(torch.int64)
    st = torch.nn.functional.one_hot(st,num_classes=3)
    st = st.permute(2, 0, 1)
    return st.to(torch.float32)

In [ ]:
board, info = env.reset()
board

In [ ]:
info['action_mask']

In [ ]:
state = preprocess_state(board)
state

In [ ]:
with torch.no_grad():
    model_output = model(state.unsqueeze(0))

In [ ]:
model_output.shape

In [ ]:
# Function to select an action
# model: the torch model to compuate action-state value (i.e., q-value)
# state: a torch tensor (3 x 8 x 8) of float32, which is output by preprocess_state
# mask: a 64-size array (np.array)
def get_greedy_epsilon(model, state, mask):
    global epsilon

    #if frame_count < epsilon_random_frames or np.random.rand(1)[0] < epsilon:
    if np.random.rand(1)[0] < epsilon:
        action = np.random.choice([ i for i in range(num_actions) if mask[i] == 1 ])
    else:
        with torch.no_grad():
            # add a batch axis
            state_tensor = state.unsqueeze(0)
            # compute the q-values
            q_values = model(state_tensor)
            # select the q-values of valid actions
            action = torch.argmax(
                q_values.squeeze() + torch.from_numpy(mask) * 100., # trick to select a valid action
                dim=0)


            #valid_q = [ (i, q_values[0][i]) for i in range(64) if mask[i] == 1 ]
            # the action of maximum q-value
            #action, _ = max(valid_q, key=lambda e: e[1])

    # decay epsilon
    epsilon -= epsilon_interval / epsilon_greedy_frames
    epsilon = max(epsilon, epsilon_min)

    return action

In [ ]:
mask = np.zeros((64,), dtype=np.int64)
mask[12] = 1.
get_greedy_epsilon(model, state, mask)

In [ ]:
def get_greedy_action(model, state, mask):
    global epsilon

    with torch.no_grad():
        state_tensor = state.unsqueeze(0) # batch dimension
        q_values = model(state_tensor)

        action = torch.argmax(
                q_values.squeeze() + torch.from_numpy(mask) * 100., # trick to select a valid action
                dim=0)

    return action

In [ ]:
# sample a batch of _batch_size from replay buffers
# return numpy.ndarrays
def sample_batch(_batch_size):
    # Get indices of samples for replay buffers
    indices = np.random.choice(range(len(done_history)), size=_batch_size, replace=False)

    state_sample = np.array([state_history[i].squeeze(0).numpy() for i in indices])
    state_next_sample = np.array([state_next_history[i].squeeze(0).numpy() for i in indices])
    rewards_sample = np.array([rewards_history[i] for i in indices], dtype=np.float32)
    action_sample = np.array([action_history[i] for i in indices])

    # action mask is the mask for the valid actions at the '''next''' state
    action_mask_sample = np.array([action_mask_history[i] for i in indices])
    done_sample = np.array([float(done_history[i]) for i in indices])

    return state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample

In [ ]:
# Function to update the Q-network
def update_network():
    # sample a batch of ...
    state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample = \
        sample_batch(batch_size)

    # Convert numpy arrays to PyTorch tensors
    state_sample = torch.tensor(state_sample, dtype=torch.float32).to(device)
    state_next_sample = torch.tensor(state_next_sample, dtype=torch.float32).to(device)
    action_sample = torch.tensor(action_sample, dtype=torch.int64).to(device)
    action_mask_sample = torch.tensor(action_mask_sample, dtype=torch.int64).to(device)
    rewards_sample = torch.tensor(rewards_sample, dtype=torch.float32).to(device)
    done_sample = torch.tensor(done_sample, dtype=torch.float32).to(device)

    # Compute the target Q-values for the states
    with torch.no_grad():
        future_rewards = model_target(state_next_sample)
        #future_rewards = future_rewards.cpu()

        # compute the q-value for the next state and the action maximizing the q-value
        # note: the action should be valid (i.e., mask is set to 1)
        max_q_values = torch.max(
            future_rewards + action_mask_sample * 100., # trick to select a valid action
            dim=1).values.detach() - 100.

        # compute the target q-value
        # if the step was final, max_q_values should not be added
        # we assume that the negative return of the opposite player is the return of next step
        # that is, G(t) = r(t+1) - g*r(t+2) + g^2*r(t+3) - g^3*r(t+4) + ...
        target_q_values = rewards_sample + gamma * max_q_values * (1. - done_sample)

    # It's forward propagation! Compute the Q-values for the taken actions
    q_values = model(state_sample)
    #q_values = q_values.cpu()
    q_values_action = q_values.gather(dim=1, index=action_sample.unsqueeze(1)).squeeze(1)

    # Compute the loss
    loss = loss_function(q_values_action, target_q_values)

    # Perform the optimization step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
board, info = env.reset()
board.shape

In [ ]:
info['action_mask'].shape

In [ ]:
state = preprocess_state(board)
state[0] # the positions of stones of the agent player

In [ ]:
action_mask = info['action_mask'].reshape((-1,))
action_mask.shape

In [ ]:
action = get_greedy_epsilon(model, state, action_mask)
print(action)

In [ ]:
board, reward, done, _, info = env.step((action // 8, action % 8))
state_next = preprocess_state(board)
action_mask = info['action_mask'].reshape((-1,))
state_next.shape

In [ ]:
reward

In [ ]:
# Save actions and states in replay buffer
action_history.append(action)
action_mask_history.append(action_mask)
state_history.append(state)
state_next_history.append(state_next)
done_history.append(done)
rewards_history.append(reward)

state = state_next

In [ ]:
# Get an action by epsilon-greedy policy
action = get_greedy_epsilon(model, state, action_mask)
action

In [ ]:
board, reward, done, _, info = env.step((action // 8, action % 8))
state_next = preprocess_state(board)
action_mask = info['action_mask'].reshape((-1,))
state_next.shape

In [ ]:
action_history.append(action)
action_mask_history.append(action_mask)
state_history.append(state)
state_next_history.append(state_next)
done_history.append(done)
rewards_history.append(reward)

In [ ]:
state_sample, state_next_sample, rewards_sample, action_sample, action_mask_sample, done_sample = \
    sample_batch(2)

In [ ]:
state_sample.shape

In [ ]:
state_next_sample.shape

In [ ]:
rewards_sample.shape

In [ ]:
# Convert numpy arrays to PyTorch tensors
state_sample = torch.tensor(state_sample, dtype=torch.float32)
state_next_sample = torch.tensor(state_next_sample, dtype=torch.float32)
action_sample = torch.tensor(action_sample, dtype=torch.int64)
action_mask_sample = torch.tensor(action_mask_sample, dtype=torch.int64)
rewards_sample = torch.tensor(rewards_sample, dtype=torch.float32)
done_sample = torch.tensor(done_sample, dtype=torch.float32)

In [ ]:
with torch.no_grad():
    future_rewards = model_target(state_next_sample.to(device))

future_rewards.shape

In [ ]:
action_sample

In [ ]:
torch.max(future_rewards, dim=1)

In [ ]:
torch.max(
        future_rewards + action_mask_sample.to(device) * 100.,
        dim=1)

In [ ]:
done_sample.unsqueeze(1)

In [ ]:
with torch.no_grad():
    future_rewards = model_target(state_next_sample.to(device))

    # compute the q-value for the next state and the action maximizing the q-value
    # note: the action should be valid (i.e., mask is set to 1)
    max_q_values = torch.max(
        future_rewards + action_mask_sample.to(device) * 100., # trick to select a valid action
        dim=1).values.detach() - 100.

    # compute the target q-value
    # if the step was final, max_q_values should not be added
    target_q_values = rewards_sample.to(device) +  max_q_values * (1. - done_sample.to(device))

target_q_values.shape

In [ ]:
# It's forward propagation! Compute the Q-values for the taken actions
q_values = model(state_sample.to(device))
q_values_action = q_values.gather(dim=1, index=action_sample.to(device).unsqueeze(1)).squeeze(1)
q_values_action.shape

In [ ]:
# Compute the loss
loss = loss_function(q_values_action, target_q_values)

In [ ]:
# Perform the optimization step
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [ ]:
#training

for _ in range(max_episodes):
    state, info = env.reset()
    state = preprocess_state(state)
    action_mask = info['action_mask'].reshape((-1,))
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        frame_count += 1

        # Select an action
        #state_cuda = state.to(device)

        # print(action_mask)

        action = get_greedy_epsilon(model, state, action_mask)


        if action < 0:
            print(action_mask)

        # Take the selected action
        state_next, reward, done, _, info = env.step((action // 8, action % 8))
        state_next = preprocess_state(state_next)
        action_mask = info['action_mask'].reshape((-1,))

        episode_reward += reward

        # Store the transition in the replay buffer
        action_history.append(action)
        action_mask_history.append(action_mask)
        state_history.append(state)
        state_next_history.append(state_next)
        rewards_history.append(reward)
        done_history.append(done)

        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            update_network()

        if frame_count % update_target_network == 0:
            model_target.load_state_dict(model.state_dict())

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del action_mask_history[:1]
            del done_history[:1]

        if done:
            break

    episode_count += 1
    episode_reward_history.append(episode_reward)

    # Update running reward to check condition for solving
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    if episode_count % 10 == 0:
        print(f"Episode: {episode_count}, Frame count: {frame_count}, Running reward: {running_reward}")

    if episode_count % 5000 == 0:
        torch.save(model, 'model.{}'.format(episode_count))
    #if running_reward > 20:
    #    print(f"Solved at episode {episode_count}!")
    #    break


torch.save(model, 'model.final')



In [ ]:
#EVAL

import time, sys
from IPython.display import clear_output

board, info = env.reset()
state = preprocess_state(board)
action_mask = info['action_mask'].reshape((-1,))
done = False
env.render()

while not done:
    action = get_greedy_action(model, state, action_mask)
    print("action: ({}, {})".format(action // 8, action % 8))
    sys.stdout.flush()

    time.sleep(1.0)
    clear_output(wait=False)
    board, reward, done, _, info = env.step((action // 8, action % 8))
    state = preprocess_state(board)
    action_mask = info['action_mask'].reshape((-1,))
    env.render()